In [1]:
import pandas as pd
import numpy as np

from yaml import safe_load
import os

from tqdm import tqdm
'''Tqdm is a Python library used to display smart progress
 bars that show the progress of your Python code execution.
 This library can also be used to see the progress of a 
 machine learning model while training the model on a very large data set.'''


'Tqdm is a Python library used to display smart progress\n bars that show the progress of your Python code execution.\n This library can also be used to see the progress of a \n machine learning model while training the model on a very large data set.'

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score,mean_absolute_error

## For Converting the yaml file into padas's dataframe

In [3]:
filename=[]
for file in os.listdir('/kaggle/input/world-t20-yaml-file/t20s_zip_file'):
    filename.append(os.path.join('/kaggle/input/world-t20-yaml-file/t20s_zip_file',file))

In [4]:
final_df=pd.DataFrame()
counter=1
for file in tqdm(filename):
    with open(file,'r') as f:
        if file.split('.')[1]!='yaml': 
            #there are some .txt file present in the zip file
            # and it cause error while json_normalize
            continue
        else:
            df=pd.json_normalize(safe_load(f))
            df['match_id']=counter
            final_df=final_df.append(df)
            counter+=1

100%|██████████| 1433/1433 [08:12<00:00,  2.91it/s]


In [5]:
pd.set_option('display.max_columns',None)

In [6]:
final_df.sample(3)

,innings,meta.data_version,meta.created,meta.revision,info.city,info.dates,info.gender,info.match_type,info.neutral_venue,info.outcome.by.wickets,info.outcome.winner,info.overs,info.player_of_match,info.teams,info.toss.decision,info.toss.winner,info.umpires,info.venue,match_id,info.match_type_number,info.outcome.by.runs,info.outcome.method,info.outcome.result,info.outcome.eliminator,info.bowl_out,info.outcome.bowl_out,info.supersubs.New Zealand,info.supersubs.South Africa
0,"[{'1st innings': {'team': 'Fiji', 'deliveries'...",0.9,2019-11-01,1,Port Vila,[2019-05-10],female,T20,NaN,8.0,Indonesia,20,[N Sitompul],"[Fiji, Indonesia]",field,Indonesia,"[Kim Cotton, ZK Shem]","Ground 1, Independence Park",308,649.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,"[{'1st innings': {'team': 'Italy', 'deliveries...",0.9,2019-06-20,1,St Peter Port,[2019-06-16],male,T20,NaN,NaN,Italy,20,NaN,"[Guernsey, Italy]",field,Guernsey,"[AG Wharf, MV Waldron]",College Field,1396,796.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,"[{'1st innings': {'team': 'Nepal', 'deliveries...",0.9,2015-07-18,1,Dublin,[2015-07-17],male,T20,1.0,8.0,Papua New Guinea,20,[A Vala],"[Nepal, Papua New Guinea]",field,Papua New Guinea,"[JD Cloete, RT Robinson]","The Village, Malahide",1363,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_backup=final_df.copy()

In [8]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1432 entries, 0 to 0
Data columns (total 28 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   innings                      1432 non-null   object 
 1   meta.data_version            1432 non-null   float64
 2   meta.created                 1432 non-null   object 
 3   meta.revision                1432 non-null   int64  
 4   info.city                    1240 non-null   object 
 5   info.dates                   1432 non-null   object 
 6   info.gender                  1432 non-null   object 
 7   info.match_type              1432 non-null   object 
 8   info.neutral_venue           328 non-null    float64
 9   info.outcome.by.wickets      662 non-null    float64
 10  info.outcome.winner          1386 non-null   object 
 11  info.overs                   1432 non-null   int64  
 12  info.player_of_match         1269 non-null   object 
 13  info.teams           

In [9]:
# drop unnecessary columns
final_df.drop(columns=[
    'meta.data_version',
    'meta.created',
    'meta.revision',
    'info.neutral_venue',
    'info.outcome.by.wickets',
    'info.match_type_number',
    'info.outcome.by.runs',
    'info.outcome.method',
    'info.outcome.result',
    'info.outcome.eliminator',
    'info.bowl_out',
    'info.outcome.bowl_out',
    'info.supersubs.New Zealand',
    'info.supersubs.South Africa'
],inplace=True)

In [10]:
final_df.shape

(1432, 14)

In [11]:
final_df['info.gender'].value_counts()

male      966
female    466
Name: info.gender, dtype: int64

In [12]:
# Taking men's match record only
final_df=final_df[final_df['info.gender']=='male']
final_df.drop(columns='info.gender',inplace=True)

In [13]:
final_df['info.match_type'].value_counts()

T20    966
Name: info.match_type, dtype: int64

In [14]:
final_df['info.overs'].value_counts()

20    963
50      3
Name: info.overs, dtype: int64

In [15]:
# Drop match having over>20
final_df=final_df[final_df['info.overs']==20]
final_df.drop(columns=['info.overs','info.match_type'],inplace=True)

In [16]:
import pickle
pickle.dump(final_df,open('(after_convert)dataset_level11.pkl','wb'))

Instead of converting yaml file into panda's dataframe everytime, make the dataset which we already
conveted, to save the time

## Working on the pickel file directly

In [17]:
matches=pickle.load(open('(after_convert)dataset_level11.pkl','rb'))

In [18]:
#matches.iloc[0]['innings']

In [19]:
matches.iloc[75]

innings                 [{'1st innings': {'team': 'Zimbabwe', 'deliver...
info.city                                                        Trinidad
info.dates                                                   [2010-02-28]
info.outcome.winner                                              Zimbabwe
info.player_of_match                                          [AG Cremer]
info.teams                                        [West Indies, Zimbabwe]
info.toss.decision                                                    bat
info.toss.winner                                                 Zimbabwe
info.umpires                                      [CR Duncan, NA Malcolm]
info.venue                               Queen's Park Oval, Port of Spain
match_id                                                              107
Name: 0, dtype: object

## Converting the dataset into the required format

In [20]:
count = 1
delivery_df = pd.DataFrame()
for index, row in matches.iterrows():
    count+=1
    ball_of_match = []
    batsman = []
    bowler = []
    runs = []
    player_of_dismissed = []
    teams = []
    batting_team = []
    match_id = []
    city = []
    venue = []
    for ball in row['innings'][0]['1st innings']['deliveries']:
        for key in ball.keys():
            match_id.append(count)
            batting_team.append(row['innings'][0]['1st innings']['team'])
            teams.append(row['info.teams'])
            ball_of_match.append(key)
            batsman.append(ball[key]['batsman'])
            bowler.append(ball[key]['bowler'])
            runs.append(ball[key]['runs']['total'])
            city.append(row['info.city'])
            venue.append(row['info.venue'])
            try:
                player_of_dismissed.append(ball[key]['wicket']['player_out'])
            except:
                player_of_dismissed.append('0')
    loop_df = pd.DataFrame({
            'match_id':match_id,
            'teams':teams,
            'batting_team':batting_team,
            'ball':ball_of_match,
            'batsman':batsman,
            'bowler':bowler,
            'runs':runs,
            'player_dismissed':player_of_dismissed,
            'city':city,
            'venue':venue
        })
    delivery_df = delivery_df.append(loop_df)

In [21]:
delivery_df.iloc[75]

match_id                                               2
teams                            [Netherlands, Scotland]
batting_team                                    Scotland
ball                                                12.3
batsman                                       KJ Coetzer
bowler                                         PW Borren
runs                                                   1
player_dismissed                                       0
city                                             Belfast
venue               Civil Service Cricket Club, Stormont
Name: 75, dtype: object

In [22]:
# Extractring the bowling team
def bowl(row):
    for team in row['teams']:
        if team != row['batting_team']:
            return team

In [23]:
delivery_df['bowling_team'] = delivery_df.apply(bowl,axis=1)

In [24]:
# Once we get bowling_team column, don't need of team column
delivery_df.drop(columns=['teams'],inplace=True)

In [25]:
delivery_df['batting_team'].value_counts()

Pakistan                    10339
South Africa                 8557
India                        8382
New Zealand                  8357
Sri Lanka                    8057
West Indies                  7243
Australia                    6984
England                      6967
Afghanistan                  5206
Bangladesh                   4881
Ireland                      4661
Zimbabwe                     4361
Netherlands                  4011
United Arab Emirates         3069
Hong Kong                    2220
Scotland                     2214
Kenya                        1819
Oman                         1743
Malaysia                     1618
Nepal                        1588
Papua New Guinea             1362
Singapore                    1360
Canada                       1231
Namibia                       989
Jersey                        983
Thailand                      751
Nigeria                       739
Bermuda                       703
Cayman Islands                619
Vanuatu       

In [26]:
# considering the top 10 teams 
teams = [
    'Australia',
    'India',
    'Bangladesh',
    'New Zealand',
    'South Africa',
    'England',
    'West Indies',
    'Afghanistan',
    'Pakistan',
    'Sri Lanka'    
]

In [27]:
delivery_df = delivery_df[delivery_df['batting_team'].isin(teams)]
delivery_df = delivery_df[delivery_df['bowling_team'].isin(teams)]

In [28]:
#delivery_df

In [29]:
# Consider only required columns
output = delivery_df[['match_id','batting_team','bowling_team','ball','runs','player_dismissed','city','venue']]

In [30]:
#output

In [31]:
output.isnull().sum()

match_id               0
batting_team           0
bowling_team           0
ball                   0
runs                   0
player_dismissed       0
city                8671
venue                  0
dtype: int64

In [32]:
output[output['city'].isnull()]['venue'].value_counts()

Dubai International Cricket Stadium        3092
Pallekele International Cricket Stadium    2066
Melbourne Cricket Ground                   1453
Sydney Cricket Ground                       749
Adelaide Oval                               498
Harare Sports Club                          372
Sharjah Cricket Stadium                     249
Sylhet International Cricket Stadium        128
Carrara Oval                                 64
Name: venue, dtype: int64

In [33]:
# to replace the null values in the city we take the venue and take first word from it 
# which mostly represent name of the city
output['city']=np.where(output['city'].isnull(),
                        output['venue'].str.split().apply(lambda x:x[0]),output['city'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [34]:
output.isnull().sum()

match_id            0
batting_team        0
bowling_team        0
ball                0
runs                0
player_dismissed    0
city                0
venue               0
dtype: int64

In [35]:
output.drop(columns="venue",inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [36]:
output['city'].value_counts()

Colombo          4208
Mirpur           3420
Johannesburg     3331
Dubai            3092
Auckland         2655
                 ... 
Victoria          123
Dharamsala        122
Potchefstroom     122
Ahmedabad         121
Carrara            64
Name: city, Length: 87, dtype: int64

In [37]:
# we are consedering the city where more than 3 matches were played.
eligible_city=output['city'].value_counts()[output['city'].value_counts()>600].index.tolist()

In [38]:
output=output[output['city'].isin(eligible_city)]

In [39]:
# calculating the current score 
output['current_score']=output.groupby('match_id').cumsum()['runs']

 Creating the ball left column

In [40]:
output['over']=output['ball'].apply(lambda x:str(x).split('.')[0])
output['balls']=output['ball'].apply(lambda x:str(x).split('.')[1])

In [41]:
output['ball_bowled']=output['over'].astype(int)*6+output['balls'].astype(int)

In [42]:
# actually there is one shortcut we use here we didn't count the extra 
# balls and therefore we get -ve number in ball left_column
# to do it first count the extra and then we get proper result but 
# this method is also gives good result 
output['ball_left']=120-output['ball_bowled']
output['ball_left']=output['ball_left'].apply(lambda x:0 if x<0 else x)

In [43]:
output[output['ball_left']==0]['ball_bowled'].unique()

array([120, 121, 122, 123])

here we saw only 2 or 3 extra bowl in entire 
dataset but how it is possible because the formula 
 which we used to calculate the ball_bowled
these 2 or 3 extra ball are belonging to the last over only

Creating the columns which cointains the wickets left

In [44]:
output['player_dismissed']=output['player_dismissed'].apply(lambda x:0 if x=='0' else 1)

In [45]:
output['player_dismissed']=output.groupby('match_id').cumsum()['player_dismissed']

In [46]:
output['wicket_left']=10-output['player_dismissed'].astype(int)

In [47]:
output.head(10)

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,balls,ball_bowled,ball_left,wicket_left
0,3,Sri Lanka,Pakistan,0.1,0,0,Abu Dhabi,0,0,1,1,119,10
1,3,Sri Lanka,Pakistan,0.2,0,0,Abu Dhabi,0,0,2,2,118,10
2,3,Sri Lanka,Pakistan,0.3,1,0,Abu Dhabi,1,0,3,3,117,10
3,3,Sri Lanka,Pakistan,0.4,0,0,Abu Dhabi,1,0,4,4,116,10
4,3,Sri Lanka,Pakistan,0.5,0,0,Abu Dhabi,1,0,5,5,115,10
5,3,Sri Lanka,Pakistan,0.6,0,0,Abu Dhabi,1,0,6,6,114,10
6,3,Sri Lanka,Pakistan,1.1,0,0,Abu Dhabi,1,1,1,7,113,10
7,3,Sri Lanka,Pakistan,1.2,4,0,Abu Dhabi,5,1,2,8,112,10
8,3,Sri Lanka,Pakistan,1.3,0,0,Abu Dhabi,5,1,3,9,111,10
9,3,Sri Lanka,Pakistan,1.4,0,0,Abu Dhabi,5,1,4,10,110,10


Creating the current run rate columns

current run rate=total_score/over

In [48]:
output['crr']=round(output['current_score']*6/output['ball_bowled'],2)

Creating the column for for last 5 over score

In [49]:
groups=output.groupby('match_id')

match_ids=output['match_id'].unique()
last_five=[]
for id in match_ids:
    #window=30 bacause we are calculating for 5 over
    last_five.extend(groups.get_group(id).rolling(window=30).sum()['runs'])

In [50]:
output['last_five']=last_five

Calculating the total score

In [51]:
final_df=output.groupby('match_id').sum()['runs'].reset_index().merge(output,on='match_id')

In [52]:
#final_df

In [53]:
final_df.rename(columns={'runs_x':'total_score','runs_y':'runs'},inplace=True)

In [54]:
# Considering the required columns
final_df=final_df[['batting_team','bowling_team','city','current_score',
                   'ball_left','wicket_left','crr','last_five','total_score']]

In [55]:
final_df.isnull().sum()

batting_team         0
bowling_team         0
city                 0
current_score        0
ball_left            0
wicket_left          0
crr                  0
last_five        12169
total_score          0
dtype: int64

In [56]:
final_df.shape

(51109, 9)

In [57]:
# droping the null values
final_df.dropna(inplace=True)
final_df.shape

(38940, 9)

In [58]:
# shuffling the data so that we can avoid the biaseness
final_df=final_df.sample(final_df.shape[0])

In [59]:
# Required dataset for model building
final_df

,batting_team,bowling_team,city,current_score,ball_left,wicket_left,crr,last_five,total_score
28488,Sri Lanka,New Zealand,Chittagong,72,49,6,6.08,30.0,119
15118,India,Pakistan,Durban,122,19,4,7.25,41.0,141
6500,England,India,Manchester,154,2,2,7.83,42.0,159
9945,India,England,Mumbai,168,8,4,9.00,62.0,177
35235,New Zealand,Australia,Christchurch,214,0,4,10.70,68.0,214
...,...,...,...,...,...,...,...,...,...
16869,Australia,Bangladesh,Barbados,125,10,4,6.82,55.0,141
551,Pakistan,New Zealand,Dubai,79,59,8,7.77,46.0,161
9378,Sri Lanka,Bangladesh,Pallekele,72,76,9,9.82,40.0,198
26453,India,New Zealand,Wellington,52,83,8,8.43,38.0,165


## Model building

In [60]:
x=final_df.drop(columns=['total_score'])
y=final_df['total_score']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1)

In [61]:
trf=ColumnTransformer([
    ('trf',OneHotEncoder(sparse=False,drop='first'),['batting_team','bowling_team','city'])
]
,remainder='passthrough')

# LinearRegression

In [62]:
pipe_LinearR=Pipeline(steps=[
    ('step1',trf),
    ('step2',StandardScaler()),
    ('step3',LinearRegression())
])

In [63]:
pipe_LinearR.fit(x_train,y_train)
y_pred_LinearR=pipe_LinearR.predict(x_test)
print("R2_score is: ",r2_score(y_test,y_pred_LinearR))
print("Mean_absolute_error: ",mean_absolute_error(y_test,y_pred_LinearR))

R2_score is:  0.6776735439087846
Mean_absolute_error:  13.553786463428386


# RandomForestRegressor

In [64]:
pipe_RFR=Pipeline(steps=[
    ('step1',trf),
    ('step2',StandardScaler()),
    ('step3',RandomForestRegressor(n_estimators=1000,max_depth=12,random_state=1))
])

In [65]:
pipe_RFR.fit(x_train,y_train)
y_pred_RFR=pipe_RFR.predict(x_test)
print("R2_score is: ",r2_score(y_test,y_pred_RFR))
print("Mean_absolute_error: ",mean_absolute_error(y_test,y_pred_RFR))

R2_score is:  0.8596133566829607
Mean_absolute_error:  8.30523629691686


# XGBoost Regressor

In [66]:
pipe=Pipeline(steps=[
    ('step1',trf),
    ('step2',StandardScaler()),
    ('step3',XGBRegressor(n_estimators=1000,learning_rate=0.1,max_depth=12,random_state=1))
])

In [67]:
pipe.fit(x_train,y_train)
y_pred=pipe.predict(x_test)
print("R2_score is:",r2_score(y_test,y_pred))
print("Mean_absolute_error: ",mean_absolute_error(y_test,y_pred))

R2_score is: 0.9882643083897508
Mean_absolute_error:  1.5161577703772782


0.9866781632205727
& 1.6847456896311206  Previopus r2_score & MAE when learning_rate=0.2 and depth 15